In [30]:
#Match_home_team_info table

from glob import glob
import pandas as pd

# Find all directories under tennis_data/
date_folders = glob('tennis_data/*')
column_home= ['match_id', 'player_id', 'name','current_rank']

# Create an empty list to store all files
files_home= []
# Loop through each date folder and load .parquet files
for folder in date_folders:
    files_home.extend(
    [f for f in glob(f'{folder}/data/raw/raw_match_parquet/home_team*.parquet') if '_score' not in f]
    )
df_home_team_info = [pd.read_parquet(file, columns=column_home) for file in files_home]
df_home_team_info = pd.concat(df_home_team_info, ignore_index=True)

In [31]:
#match_away_team_info table
column_away= ['match_id', 'player_id','name','current_rank']
files_away= []
# Loop through each date folder and load .parquet files
for folder in date_folders:
    files_away.extend(
    [f for f in glob(f'{folder}/data/raw/raw_match_parquet/away_team*.parquet') if '_score' not in f]
    )
df_away_team_info = [pd.read_parquet(file, columns=column_away) for file in files_home]
df_away_team_info = pd.concat(df_away_team_info, ignore_index=True)

In [4]:
# Match_event_info table
files_event = []
column_event= ['match_id','winner_code']
# Loop through each date folder and load .parquet files
for folder in date_folders:
    files_event.extend(
        glob(f'{folder}/data/raw/raw_match_parquet/event*.parquet')
    )
event_info = [pd.read_parquet(file, columns=column_event) for file in files_event]
event_info = pd.concat(event_info, ignore_index=True)

In [57]:
# Merge all necessary tables: event info + home/away player info
df1 = (
    event_info
    .merge(df_home_team_info, on='match_id', suffixes=('', '_home'))
    .merge(df_away_team_info, on='match_id', suffixes=('_home', '_away'))
)

home_wins = df1[(df1['current_rank_away']<=10) & (df1['winner_code']==1)]
away_wins = df1[(df1['current_rank_home']<=10) & (df1['winner_code']==2)]
wins = pd.concat([
    home_wins[['player_id_home']].rename(columns={'player_id_home': 'player_id'}),
    away_wins[['player_id_away']].rename(columns={'player_id_away': 'player_id'})
])
wins
home_matches = df1[df1['current_rank_away'] <= 10][['player_id_home']].rename(columns={'player_id_home': 'player_id'})
away_matches = df1[df1['current_rank_home'] <= 10][['player_id_away']].rename(columns={'player_id_away': 'player_id'})
matches = pd.concat([home_matches, away_matches])
